<a href="https://colab.research.google.com/github/apoorvapu/data_science/blob/main/NLP_languageTranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**NLP**: **(2 methods: (1) fine-tuning LLM and (2) model training from scratch)**

# language translation

For fine-tuning Meta-llama2 for language translation with low memory,you can use techniques like LoRA, QLoRA, 8-bit/4-bit quantization, and Retrieval-Augmented Generation (RAG).

In [1]:
from google.colab import drive
drive.mount('/content/drive')
# Read the API key from the file
with open("/content/drive/My Drive/hf_token.txt", "r") as f:
    hf_token = f.read().strip()

from huggingface_hub import login
login(hf_token)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Read the API key from the file
with open('/content/drive/MyDrive/wandb_key.txt', 'r') as f:
    wandb_key = f.read().strip()

# Set the W&B API key
import os
os.environ["WANDB_API_KEY"] = wandb_key

In [3]:
!pip install datasets
!pip install bitsandbytes accelerate transformers

In [4]:
import pandas as pd
from transformers import LlamaForCausalLM, LlamaTokenizer, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
import torch, gc
from datasets import load_dataset
torch.cuda.empty_cache()
gc.collect()

90

In [5]:
# Check if CUDA (GPU support) is available
cuda_available = torch.cuda.is_available()
print("CUDA Available:", cuda_available)

# Get the name of the GPU being used
gpu_name = torch.cuda.get_device_name(0) if cuda_available else "No GPU"
print("GPU Name:", gpu_name)

CUDA Available: True
GPU Name: Tesla T4


In [6]:
# Load dataset
dataset = load_dataset("rahular/itihasa")  # Example: sanskrit-eng translation

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

In [8]:
'''
# Load tokenizer and model
model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = LlamaTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
'''

'\n# Load tokenizer and model\nmodel_name = "meta-llama/Llama-2-7b-hf"\ntokenizer = LlamaTokenizer.from_pretrained(model_name)\nmodel = LlamaForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)\n'

In [9]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50", quantization_config=quantization_config)


`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [10]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 75162
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 6149
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 11722
    })
})


In [11]:
dataset['train'][0]

{'translation': {'en': 'The ascetic Vālmīki asked Nārada, the best of sages and foremost of those conversant with words, ever engaged in austerities and Vedic studies.',
  'sn': 'ॐ तपः स्वाध्यायनिरतं तपस्वी वाग्विदां वरम्। नारदं परिपप्रच्छ वाल्मीकिर्मुनिपुङ्गवम्॥'}}

In [12]:
import gc
train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]
del dataset
gc.collect()

60

In [13]:
train_dataset[0]

{'translation': {'en': 'The ascetic Vālmīki asked Nārada, the best of sages and foremost of those conversant with words, ever engaged in austerities and Vedic studies.',
  'sn': 'ॐ तपः स्वाध्यायनिरतं तपस्वी वाग्विदां वरम्। नारदं परिपप्रच्छ वाल्मीकिर्मुनिपुङ्गवम्॥'}}

In [14]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    # Extract English ('en') and Sanskrit ('sn') texts from the 'translation' field in the list
    inputs = tokenizer([example['en'] for example in examples['translation']], padding="max_length", truncation=True, max_length=64)
    labels = tokenizer([example['sn'] for example in examples['translation']], padding="max_length", truncation=True, max_length=64)

    # Add the labels as the 'input_ids' of the Sanskrit tokens
    inputs['labels'] = labels['input_ids']
    return inputs


In [15]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/75162 [00:00<?, ? examples/s]

Map:   0%|          | 0/6149 [00:00<?, ? examples/s]

Map:   0%|          | 0/11722 [00:00<?, ? examples/s]

In [16]:
torch.cuda.empty_cache()
gc.collect()

64

In [17]:
# Define LoRA configuration
lora_config = LoraConfig(
    r=4,  # Rank for LoRA
    lora_alpha=16,  # Scaling factor for LoRA
    target_modules=["q_proj", "v_proj"],  # Modules to apply LoRA to
    lora_dropout=0.1,  # Dropout for LoRA layers
    task_type=TaskType.SEQ_2_SEQ_LM,
    bias="none",  # Freeze biases
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)
device = "cuda" if torch.cuda.is_available() else "cpu"
# Move your model to the GPU
model.to(device)

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): MBartForConditionalGeneration(
      (model): MBartModel(
        (shared): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
        (encoder): MBartEncoder(
          (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
          (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
          (layers): ModuleList(
            (0-11): 12 x MBartEncoderLayer(
              (self_attn): MBartSdpaAttention(
                (k_proj): lora.Linear4bit(
                  (base_layer): Linear4bit(in_features=1024, out_features=1024, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1024, out_features=4, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=4, 

In [18]:
torch.cuda.empty_cache()

# Set up the TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",  # Directory to save results and model checkpoints
    evaluation_strategy="steps",  # Evaluation strategy (every `eval_steps` steps)
    save_strategy="steps",  # Save checkpoint every `save_steps`
    save_steps=500,  # Save model every 500 steps (adjust as needed)
    save_total_limit=2,  # Keep the latest 3 checkpoints
    logging_dir="./logs",  # Where to save logs
    logging_steps=100,  # Log every 100 steps
    per_device_train_batch_size=4,  # Adjust depending on your GPU memory
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,  # Simulate larger batch sizes if needed
    fp16=True,  # Enable mixed precision for memory optimization
    load_best_model_at_end=True,  # Automatically load the best model at the end of training
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Clear CUDA cache
torch.cuda.empty_cache()
gc.collect()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-18-ceb48633fbe4>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


341

In [19]:
# fine-tune model
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: apoorvapu (apoorvapu-ohio-state-buckeyes) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
model.save_pretrained("./fine_tuned_llama2")
tokenizer.save_pretrained("./fine_tuned_llama2")


In [ ]:
model.predict()